## 中国大学排名爬取

爬取网站为 url=<http://www.zuihaodaxue.com/zuihaodaxuepaiming2019.html>

定向爬取之前要验证爬取可行性：
1. 定向爬取内容是否在html中，而不是js编写的动态的
2. 检查网站robots协议

右键查看网页源代码
![爬取内容html检查](https://user-images.githubusercontent.com/62495140/79990684-eacd7f00-84e3-11ea-8acc-66707d809fae.jpg)

![robots](https://user-images.githubusercontent.com/62495140/79990739-fc168b80-84e3-11ea-8311-90f833af8c4c.png)

In [8]:
import requests
from bs4 import BeautifulSoup
import bs4

In [12]:
# 1. 爬取大学排名网页内容
def getHTMLText(url):
    try: 
        r = requests.get(url,timeout=30)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ""

    
# 2. 提取网页内容(html)中的信息到列表(ulist)中
def fillUnivList(ulist, html):
    soup = BeautifulSoup(html, 'html.parser')
    for tr in soup.find('tbody').children:
        # 子节点中包含字符串节点，要对tr进行type类型判断
        if isinstance(tr, bs4.element.Tag):  # bs4类型要单独引用bs4库
            tds = tr('td') # 相当于<tag>.find_all(name),返回name为‘td’的标签列表
            ulist.append([tds[0].string, tds[1].string, tds[3].string])  # 提取‘td’标签的NavigableString内容，即排名，学校名称，总分

            
# 3. 格式化输出结果, python常用.format
def printUnivList(ulist, num):
    #打印表头
    print("{:^10}\t{:^6}\t{:^10}".format("排名","学校名称","总分")) # {:^10}居中对齐，该位置宽度占10个字符
    #打印表格
    for i in range(num):
        u = ulist[i]
        print("{:^10}\t{:^6}\t{:^10}".format(u[0],u[1],u[2]))
        
        
def main():
    uinfo = []
    url = 'http://www.zuihaodaxue.com/zuihaodaxuepaiming2019.html'
    html = getHTMLText(url)
    fillUnivList(uinfo, html)
    printUnivList(uinfo, 20) # 打印前20个
    

    
if __name__ == '__main__':
    main()

    排名    	 学校名称 	    总分    
    1     	 清华大学 	   94.6   
    2     	 北京大学 	   76.5   
    3     	 浙江大学 	   72.9   
    4     	上海交通大学	   72.1   
    5     	 复旦大学 	   65.6   
    6     	中国科学技术大学	   60.9   
    7     	华中科技大学	   58.9   
    7     	 南京大学 	   58.9   
    9     	 中山大学 	   58.2   
    10    	哈尔滨工业大学	   56.7   
    11    	北京航空航天大学	   56.3   
    12    	 武汉大学 	   56.2   
    13    	 同济大学 	   55.7   
    14    	西安交通大学	   55.0   
    15    	 四川大学 	   54.4   
    16    	北京理工大学	   54.0   
    17    	 东南大学 	   53.6   
    18    	 南开大学 	   52.8   
    19    	 天津大学 	   52.3   
    20    	华南理工大学	   52.0   


### 第二版——解决格式化输出中的中文对齐问题

format()函数的约定
![format](https://user-images.githubusercontent.com/62495140/79994095-1ce0e000-84e8-11ea-9bc6-51339c343ada.jpg)

当填入内容的长度小于设定宽度(第4项)，format就会自动按照所给填充(第2项)进行填充补全

但是默认填充格式为西文，西文与中文长度不同，所以会造成中文对齐问题。

解决方法为在format中利用中文空格进行填充

In [15]:
def getHTMLText(url):
    try: 
        r = requests.get(url,timeout=30)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ""

    

def fillUnivList(ulist, html):
    soup = BeautifulSoup(html, 'html.parser')
    for tr in soup.find('tbody').children:
        if isinstance(tr, bs4.element.Tag): 
            tds = tr('td') 
            ulist.append([tds[0].string, tds[1].string, tds[3].string])

            
# 修改
def printUnivList(ulist, num):
    
    tplt = "{0:^10}\t{1:{3}^10}\t{2:^10}"  
    #{1:{3}^10}中的1表示输出的该位置为format()函数的第1个参数（从0开始）
    # {1:{3}^10}中的{3}指用format()函数中的第3个参数进行填充，即利用chr(12288)填充
    # 中文空格的UTF-8编码：chr(12288)
    
    
    #打印表头
    print(tplt.format("排名","学校名称","总分", chr(12288))) 
    #打印表格
    for i in range(num):
        u = ulist[i]
        print(tplt.format(u[0],u[1],u[2],chr(12288)))
        
        
def main():
    uinfo = []
    url = 'http://www.zuihaodaxue.com/zuihaodaxuepaiming2019.html'
    html = getHTMLText(url)
    fillUnivList(uinfo, html)
    printUnivList(uinfo, 20) 
    

    
if __name__ == '__main__':
    main()

    排名    	　　　学校名称　　　	    总分    
    1     	　　　清华大学　　　	   94.6   
    2     	　　　北京大学　　　	   76.5   
    3     	　　　浙江大学　　　	   72.9   
    4     	　　上海交通大学　　	   72.1   
    5     	　　　复旦大学　　　	   65.6   
    6     	　中国科学技术大学　	   60.9   
    7     	　　华中科技大学　　	   58.9   
    7     	　　　南京大学　　　	   58.9   
    9     	　　　中山大学　　　	   58.2   
    10    	　哈尔滨工业大学　　	   56.7   
    11    	　北京航空航天大学　	   56.3   
    12    	　　　武汉大学　　　	   56.2   
    13    	　　　同济大学　　　	   55.7   
    14    	　　西安交通大学　　	   55.0   
    15    	　　　四川大学　　　	   54.4   
    16    	　　北京理工大学　　	   54.0   
    17    	　　　东南大学　　　	   53.6   
    18    	　　　南开大学　　　	   52.8   
    19    	　　　天津大学　　　	   52.3   
    20    	　　华南理工大学　　	   52.0   
